# Work in progress...

In [1]:
module load daint-gpu
module switch PrgEnv-gnu PrgEnv-cray
module load perftools-lite

In [2]:
make clean
make VERSION=orig
make VERSION=counters
make VERSION=kblocking
make VERSION=inlining
make VERSION=inlining-v2

rm -f -rf *~ *.o *.mod *.MOD *.i *.x *.x+orig *.x+[0-9]* core.* *.out
ftn -eZ -ffree -N255 -ec -eC -eI -eF -c m_utils.F90
ftn -eZ -ffree -N255 -ec -eC -eI -eF -c stencil2d-orig.F90
ftn -eZ -ffree -N255 -ec -eC -eI -eF m_utils.o stencil2d-orig.o -o stencil2d-orig.x
INFO: creating the CrayPat-instrumented executable 'stencil2d-orig.x' (lite-samples) ...OK
cp stencil2d-orig.x stencil2d.x
ftn -eZ -ffree -N255 -ec -eC -eI -eF -c stencil2d-counters.F90
ftn -eZ -ffree -N255 -ec -eC -eI -eF m_utils.o stencil2d-counters.o -o stencil2d-counters.x
INFO: creating the CrayPat-instrumented executable 'stencil2d-counters.x' (lite-samples) ...OK
cp stencil2d-counters.x stencil2d.x
ftn -eZ -ffree -N255 -ec -eC -eI -eF -c stencil2d-kblocking.F90
ftn -eZ -ffree -N255 -ec -eC -eI -eF m_utils.o stencil2d-kblocking.o -o stencil2d-kblocking.x
INFO: creating the CrayPat-instrumented executable 'stencil2d-kblocking.x' (lite-samples) ...OK
cp stencil2d-kblocking.x stencil2d.x
ftn -eZ -ffree -N255 -ec -eC -eI -e

In [3]:
for tag in orig kblocking inlining inlining-v2 ; do
    echo "# nx ny nz runtime[s]" > data_${tag}.py
    echo "data_${tag} = np.array( [ \\" >> data_${tag}.py
    nz=64
    for nx in 16 32 48 64 96 128 ; do
        for ny in 16 32 48 64 96 128 ; do
            sleep 2
            echo "=> running nx=${nx} ny=${ny} tag=${tag}"
            srun -n 12 ./stencil2d-${tag}.x+orig -nx ${nx} -ny ${ny} -nz ${nz} -num_iter 128 2>&1 1> /tmp/stencil2d-counters.$$.out
            runtime=`cat /tmp/stencil2d-counters.$$.out | grep " work " | awk '{print $6}' | tr -d '\n'`
            echo "[${nx}, ${ny}, ${nz}, ${runtime}], \\"  >> data_${tag}.py
            /bin/rm -f /tmp/stencil2d-counters.*.out 2>/dev/null 1>/dev/null
        done
    done
    echo "] )" >> data_${tag}.py
done

=> running nx=16 ny=16 tag=orig
=> running nx=16 ny=32 tag=orig
=> running nx=16 ny=48 tag=orig
=> running nx=16 ny=64 tag=orig
=> running nx=16 ny=96 tag=orig
=> running nx=16 ny=128 tag=orig
=> running nx=32 ny=16 tag=orig
=> running nx=32 ny=32 tag=orig
=> running nx=32 ny=48 tag=orig
=> running nx=32 ny=64 tag=orig
=> running nx=32 ny=96 tag=orig
=> running nx=32 ny=128 tag=orig
=> running nx=48 ny=16 tag=orig
=> running nx=48 ny=32 tag=orig
=> running nx=48 ny=48 tag=orig
=> running nx=48 ny=64 tag=orig
=> running nx=48 ny=96 tag=orig
=> running nx=48 ny=128 tag=orig
=> running nx=64 ny=16 tag=orig
=> running nx=64 ny=32 tag=orig
=> running nx=64 ny=48 tag=orig
=> running nx=64 ny=64 tag=orig
=> running nx=64 ny=96 tag=orig
=> running nx=64 ny=128 tag=orig
=> running nx=96 ny=16 tag=orig
=> running nx=96 ny=32 tag=orig
=> running nx=96 ny=48 tag=orig
=> running nx=96 ny=64 tag=orig
=> running nx=96 ny=96 tag=orig
=> running nx=96 ny=128 tag=orig
=> running nx=128 ny=16 tag=orig
=>